In [2]:
!pip install torchvision==0.14

   ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
   -------------------------------------- - 1.0/1.1 MB 10.1 MB/s eta 0:00:01
   ---------------------------------------- 1.1/1.1 MB 3.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/167.3 MB ? eta -:--:--
   ---------------------------------------- 1.0/167.3 MB 8.4 MB/s eta 0:00:20
    --------------------------------------- 2.1/167.3 MB 4.5 MB/s eta 0:00:37
    --------------------------------------- 2.1/167.3 MB 4.5 MB/s eta 0:00:37
    --------------------------------------- 2.9/167.3 MB 3.3 MB/s eta 0:00:50
    --------------------------------------- 3.1/167.3 MB 3.5 MB/s eta 0:00:48
    --------------------------------------- 3.1/167.3 MB 3.5 MB/s eta 0:00:48
   - -------------------------------------- 4.2/167.3 MB 2.8 MB/s eta 0:00:58
   - -------------------------------------- 5.2/167.3 MB 3.0 MB/s eta 0:00:54
   - -------------------------------------- 6.0/167.3 MB 3.1 MB/s eta 0:00:53
   - 

  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.5.1 requires torch==2.5.1, but you have torch 1.13.0 which is incompatible.

[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
from ultralytics import RTDETR

In [13]:
import cv2
print(cv2.__version__)

4.8.0


In [1]:
import torch

In [2]:
torch.__version__

'2.5.1+cu121'

In [3]:
torch.cuda.is_available()

True

In [21]:
import cv2
from ultralytics import RTDETR

model = RTDETR(r'D:\ery\School\Jupyter Notebooks\runs\detect\RT-DETR_model\weights\best.pt')
model = model.to('cuda')
model.info()

input_video_path = r'D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4'
output_video_path = r'D:\ery\School\Jupyter Notebooks\datasets\sample_vid_output.mp4'

cap = cv2.VideoCapture(input_video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

fourcc = cv2.VideoWriter_fourcc(*'mp4v')  # or use 'avc1' for H.264 codec
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

results = model(input_video_path, stream=True)

window_name = "RT-DETR Prediction"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 800, 600)  

for result in results:
    frame = result.plot()  
    
    out.write(frame)
    
    cv2.imshow(window_name, frame)

    if cv2.waitKey(1) == ord('q'):
        break

out.release()
cv2.destroyAllWindows()

print(f"Video saved to: {output_video_path}")

rt-detr-l summary: 457 layers, 32,820,461 parameters, 0 gradients, 108.0 GFLOPs

video 1/1 (frame 1/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 42 cars, 7 trucks, 130.9ms
video 1/1 (frame 2/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 36 cars, 5 trucks, 129.2ms
video 1/1 (frame 3/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 41 cars, 6 trucks, 59.1ms
video 1/1 (frame 4/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 44 cars, 7 trucks, 54.6ms
video 1/1 (frame 5/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 48 cars, 4 trucks, 57.3ms
video 1/1 (frame 6/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 46 cars, 5 trucks, 60.0ms
video 1/1 (frame 7/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 42 cars, 5 trucks, 56.8ms
video 1/1 (frame 8/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 44 cars, 7 trucks, 57.3

## prediction with sumo simulation data

In [13]:
import traci
import cv2
from ultralytics import RTDETR
from collections import defaultdict

In [6]:
model = RTDETR(r'D:\ery\School\Jupyter Notebooks\runs\detect\RT-DETR_model\weights/best.pt')
model = model.to('cuda')

In [7]:
input_video_path = r'D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4'
output_video_path = r'D:\ery\School\Jupyter Notebooks\datasets\sample_vid_tracked.mp4'

In [8]:
cap = cv2.VideoCapture(input_video_path)
fps = int(cap.get(cv2.CAP_PROP_FPS))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

In [9]:
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

In [14]:
detected_ids = defaultdict(int)
total_count = 0

In [16]:
results = model.track(
    input_video_path,
    stream=True,  
    persist=True,  
    tracker="bytetrack.yaml"  
)

In [17]:
window_name = "RT-DETR + ByteTrack"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 800, 600)

for frame_idx, result in enumerate(results):
    frame = result.plot()  # Draw detections + tracking IDs
    
    if result.boxes.id is not None:
        track_ids = result.boxes.id.cpu().numpy().astype(int)
        
        for track_id in track_ids:
            if track_id not in detected_ids:
                total_count += 1  
            detected_ids[track_id] = frame_idx  
    
    cv2.putText(
        frame,
        f"Total Objects: {total_count}",
        (20, 50),
        cv2.FONT_HERSHEY_SIMPLEX,
        1,
        (0, 255, 0),
        2,
    )
    
    out.write(frame)
    
    cv2.imshow(window_name, frame)
    
    if cv2.waitKey(1) == ord('q'):
        break

# Release resources
out.release()
cv2.destroyAllWindows()

print(f"Tracking results saved to: {output_video_path}")
print(f"Total unique objects detected: {total_count}")


video 1/1 (frame 1/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 42 cars, 7 trucks, 50.8ms
video 1/1 (frame 2/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 38 cars, 6 trucks, 54.0ms
video 1/1 (frame 3/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 38 cars, 6 trucks, 63.3ms
video 1/1 (frame 4/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 34 cars, 6 trucks, 118.8ms
video 1/1 (frame 5/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 35 cars, 4 trucks, 54.4ms
video 1/1 (frame 6/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 31 cars, 5 trucks, 58.6ms
video 1/1 (frame 7/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 32 cars, 5 trucks, 57.9ms
video 1/1 (frame 8/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp4: 640x640 30 cars, 6 trucks, 54.0ms
video 1/1 (frame 9/307) D:\ery\School\Jupyter Notebooks\datasets\sample_vid.mp

## Running model with saved counted object detected

In [1]:
import cv2
import csv
from datetime import datetime
from collections import defaultdict
from ultralytics import RTDETR

model = RTDETR(r'D:\ery\School\Jupyter Notebooks\runs\detect\RT-DETR_retrain\weights\rtdetr_weights.pt')
model = model.to('cuda')

input_video_path = r'D:\ery\School\Jupyter Notebooks\datasets\test (online-video-cutter.com).mp4'
output_video_path = r'D:\ery\School\Jupyter Notebooks\datasets\test (online-video-cutter.com)_tracked.mp4'
csv_file_path = r'D:\ery\School\Jupyter Notebooks\datasets\test_object_counts.csv'

# part for configu
cap = cv2.VideoCapture(input_video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
cap.release()

# part for encoding the video output
fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

unique_vehicles = set() 
unique_pedestrians = set() 
frame_count = 0

csv_header = ["Timestamp", "Frame", "Total_Vehicles", "Total_Pedestrians"]

with open(csv_file_path, mode='w', newline='') as csv_file:
    writer = csv.writer(csv_file)
    writer.writerow(csv_header)

results = model.track(
    input_video_path,
    stream=True,
    persist=True,
    tracker="bytetrack.yaml",
    conf=0.5
)

window_name = "Pedestrian & Vehicle Tracker"
cv2.namedWindow(window_name, cv2.WINDOW_NORMAL)
cv2.resizeWindow(window_name, 800, 600)

for frame_idx, result in enumerate(results):
    frame = result.plot()
    frame_count += 1
    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")

    if result.boxes.id is not None:
        track_ids = result.boxes.id.cpu().numpy().astype(int)
        class_ids = result.boxes.cls.cpu().numpy().astype(int)

        for track_id, class_id in zip(track_ids, class_ids):
            class_name = model.names[class_id]

            if class_name == "car" or class_name == "truck":
                if track_id not in unique_vehicles:
                    unique_vehicles.add(track_id)
            elif class_name == "person":
                if track_id not in unique_pedestrians:
                    unique_pedestrians.add(track_id)

    cv2.putText(frame, f"Vehicles: {len(unique_vehicles)}", (20, 50), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
    cv2.putText(frame, f"Pedestrians: {len(unique_pedestrians)}", (20, 100), 
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

    out.write(frame)

    with open(csv_file_path, mode='a', newline='') as csv_file:
        writer = csv.writer(csv_file)
        writer.writerow([
            current_time,
            frame_idx,
            len(unique_vehicles),
            len(unique_pedestrians)
        ])

    cv2.imshow(window_name, frame)
    if cv2.waitKey(1) == ord('q'):
        break

out.release()
cv2.destroyAllWindows()

print(f"Results saved to: {output_video_path}")
print(f"CSV data saved to: {csv_file_path}")
print(f"Final counts - Vehicles: {len(unique_vehicles)}, Pedestrians: {len(unique_pedestrians)}")


video 1/1 (frame 1/92) D:\ery\School\Jupyter Notebooks\datasets\test (online-video-cutter.com).mp4: 640x640 2 bicycles, 1 car, 3 motorcycles, 13 persons, 59.7ms
video 1/1 (frame 2/92) D:\ery\School\Jupyter Notebooks\datasets\test (online-video-cutter.com).mp4: 640x640 1 bicycle, 1 motorcycle, 15 persons, 74.2ms
video 1/1 (frame 3/92) D:\ery\School\Jupyter Notebooks\datasets\test (online-video-cutter.com).mp4: 640x640 3 bicycles, 1 bus, 2 motorcycles, 14 persons, 80.9ms
video 1/1 (frame 4/92) D:\ery\School\Jupyter Notebooks\datasets\test (online-video-cutter.com).mp4: 640x640 4 motorcycles, 13 persons, 69.1ms
video 1/1 (frame 5/92) D:\ery\School\Jupyter Notebooks\datasets\test (online-video-cutter.com).mp4: 640x640 1 bicycle, 5 motorcycles, 11 persons, 71.1ms
video 1/1 (frame 6/92) D:\ery\School\Jupyter Notebooks\datasets\test (online-video-cutter.com).mp4: 640x640 1 bicycle, 5 motorcycles, 15 persons, 124.7ms
video 1/1 (frame 7/92) D:\ery\School\Jupyter Notebooks\datasets\test (online